<h2><center>NLP Text Classification</h2>

## I. Introduction

### 1.1 Domain-specific area
This project provides an analysis of textual data on Twitter to accurately detect and classify threatening or harmful content using sentiment analysis techniques. This would provide the cybersecurity industry a tool that takes in a corpus of text for training to develop a strong detection system.

### 1.2 Objectives
Due to popular algorithms being centered around the detection of cyberbullying on social media (Cynthia Van Hee et al., 2018), it is important for this project to widen the scope of detection. While the general detection algorithms focus mainly on terrorism and cyberbullying, it is a known fact that cybersecurity encompasses more than those 2 focuses. (Khairy, Mahmoud and Abd-El-Hafeez, 2021) While full security and safety of users cannot be ensured, making these adjustments would contribute valuable insights for future development.

### 1.3 Dataset
To begin this project, an extensive amount of textual data corpora is required. After researching large datasets of Tweets, Sentiment140 Kaggle was proven to be the best for this project. With 1.6 million tweets extracted using the Twitter API, the authors have categorised each tweet to have either a positive, neutral or negative sentiment, which is beneficial for the algorithm in categorising harmful texts.

The dataset consists of the target (defined as the sentiment of the text), the tweet IDs, date, flags (possible queries, which would be removed in the initialisation phase of extracting the data), the username, and the text of the tweet.

### 1.4 Evaluation methodology

## II. Implementation

### 2.1 Pre-processing
(writeup not needed)
<br>Convert/store the dataset locally and preprocess the data. Describe the text representation
(e.g., bag of words, word embedding, etc.) and any pre-processing steps you have applied
and why they were needed (e.g. tokenization, lemmatization). Describe the vocabulary and
file type/format, e.g. CSV file.

#### Acquiring dataset
The dataset on the collection of Tweets were acquired from Kaggle by downloading the CSV file. The author of this dataset is Μαριος Μιχαηλιδης KazAnova. The code for importing the dataset is shown below:

#### 2.1.1 Importing libraries
- <b>pandas library</b> was imported to process and handle datasets in Python. It is used to help write and read from CSV files while handling real-world messy data and processing them into a proper format

- <b>numpy library</b> was imported to handle calculations and use numpy arrays for statistical calculations

- <b>matplotlib library</b> was imported to plot the data and represent it graphically

- <b>os library</b> was imported to have a way of using the operating system dependent functionalities, more specifically to save the dataset as a CSV

- <b>string library</b> contains all ASCII characters considered as whitespace to aid in stopwords removal

- <b>stopwords library</b> was imported to have a library of the most common words in data to aid in stopwords removal

In [1]:
# dataframes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# text processing and analysis
import nltk
import string
from nltk.corpus import stopwords

#### 2.1.2 Creating helper functions
- <b>displaysetsH</b> takes in a list of datasets and an optional number of rows to display the head of each dataset

- <b>displaysetsT</b> takes in a list of datasets and an optional number of rows to display the tail of each dataset

- <b>resetidx</b> takes in a list of datasets to reset the indexes of each dataset

In [2]:
def displaysetsH(datasets, amt = 5):
    for dataset in datasets:
        display(dataset.head(amt))
        
def displaysetsT(datasets, amt = 5):
    for dataset in datasets:
        display(dataset.head(amt))
        
def resetidx(datasets):
    for dataset in datasets:
        dataset.reset_index(drop = True, inplace = True)

#### 2.1.3 Importing dataset
Due to the dataset being too large for analysis, we will analyse the first 1000 textual data corpora.

In [3]:
df = pd.read_csv("datasets/sentiment140_dataset.csv", nrows = 1000)
df

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
995,0,1468055266,Mon Apr 06 23:28:41 PDT 2009,NO_QUERY,ohmigosh_dusti,@t_wolfe i miss u too. i'm totally comin back...
996,0,1468055472,Mon Apr 06 23:28:43 PDT 2009,NO_QUERY,tiphaniebrooke,@sniffinglue ohhh. I love it. ps I'm sad we di...
997,0,1468055604,Mon Apr 06 23:28:45 PDT 2009,NO_QUERY,rinahannah,And somehow I still end up in this place
998,0,1468055791,Mon Apr 06 23:28:49 PDT 2009,NO_QUERY,ecjc,"@kisluvkis oh that is very sad, poor boy."


#### 2.1.4 Adding headers
Based on analysing tweets_df.head(), it is seen that the dataset does not have any headers. As such, the first process would be to add the headers to aid in future analysis. The columns 'tweet_id', 'date', 'flag' and 'user' will then be removed as the project focus is on the sentiment analysis. This modified dataset will then be stored as a new CSV file.

(<i>the headers are modified based on looking at the contents from Kaggle</i>)

In [4]:
headers = ['sentiment', 'tweet_id', 'date', 'flag', 'user', 'tweet']
tweets_df = pd.read_csv("datasets/sentiment140_dataset.csv", nrows = 1000, names = headers)
columns_to_drop = ['tweet_id', 'date', 'flag', 'user']
tweets_df.drop(columns = columns_to_drop, inplace = True)
tweets_df

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
995,0,@dkoenigs thanks man. I'm so very grateful. ...
996,0,@t_wolfe i miss u too. i'm totally comin back...
997,0,@sniffinglue ohhh. I love it. ps I'm sad we di...
998,0,And somehow I still end up in this place


In [5]:
file_path = 'datasets/expanded_sentiment140_dataset.csv'

if not os.path.exists(file_path):
    tweets_df.to_csv(file_path, index = False)
    print('File saved successfully.')
else:
    print('File already exists.')
   
tweets_df = pd.read_csv("datasets/expanded_sentiment140_dataset.csv")

File already exists.


#### 2.1.5 Checking for duplicates and null entries
To ensure that the analysis is beneficial, all 1000 entries that were chosen should be unique. To do so, a 'duplicated' column will be added to the a temporary copy of the dataset which is the output of df.duplicated() and we will print only columns where the 'duplicated' column is True.

Based on the output, it is seen that there are no duplicated Tweets.

After ensuring that all entries are unique, a check that all entries do not have NULL entries will be done. This is because sentiment analysis cannot be done on empty texts.

Based on the output, it is seen that there are no NULL entries.

In [6]:
dupe_checker = tweets_df.copy()
duplicates = dupe_checker.duplicated()
dupe_checker['duplicated'] = duplicates
duplicated = dupe_checker[dupe_checker['duplicated'] == True]
duplicated

,sentiment,tweet,duplicated


In [7]:
null_checker = tweets_df.copy()
null_checker.isnull().sum()

sentiment    0
tweet        0
dtype: int64

#### 2.1.6 Basic statistics
To ensure that the data is good for analysis, the amount of positive, neutral and negative sentiments should be as balanced as possible (sentiment values: 0 = negative, 2 = neutral, 4 = positive)

Based on the output, it is seen that the dataset has been skewed. However, due to the focus on the project being the detection of threatful comments, the skewed focus on negative sentiments should be sufficient enough for detection.

** Second option: Based on the output, it is seen that the dataset has been skewed. Due to this, a new collation of the first and last 1000 entries will be used.

In [8]:
temp = tweets_df.copy()
print("Number of positive sentiments:", temp[temp['sentiment'] == 4].count())
print("Number of neutral sentiments:", temp[temp['sentiment'] == 2].count())
print("Number of negative sentiments:", temp[temp['sentiment'] == 0].count())

Number of positive sentiments: sentiment    0
tweet        0
dtype: int64
Number of neutral sentiments: sentiment    0
tweet        0
dtype: int64
Number of negative sentiments: sentiment    1000
tweet        1000
dtype: int64


#### 2.1.7 Basic text processing
To begin the process of analysing the text, we would require conducting basic text processing methods.

- <b>Removing stop words</b>: In human language, it is very common for stop words to be present. These words, including determiners (eg: the, a, this, my), conjunctions (eg: and, or, nor, but, whereas) and prepositions (eg: against, along, at, before), are used to connect thoughts and speech to form grammatically accurate sentences or structural cohesion. While important during communication amongst one another, they do not carry importance or sentiments that would be valuable to this project, thus introducing noise. The removal would help to streamline the process to focus on words that would contribute more meaning to the sentiment of Tweets.
Convert/store the dataset locally and preprocess the data. Describe the text representation (e.g., bag of words, word embedding, etc.) and any pre-processing steps you have applied and why they were needed (e.g. tokenization, lemmatization). Describe the vocabulary and file type/format, e.g. CSV file.

In [9]:
# Downloading stopword corpus
nltk.download("stopwords")
# Get stopword list
stop_words = set(stopwords.words("english"))
print("stop_words", stop_words)

stop_words {"wouldn't", 'out', 'an', 'having', 'its', 'the', 'there', 'was', 'over', 'only', 'm', "doesn't", 'below', 'each', 'her', "hadn't", 'about', 'for', 'too', 'by', 'or', 'are', 'while', 'aren', 'wouldn', 'should', 'shan', 'been', 'ourselves', 'themselves', 'hers', "mustn't", 'between', 'their', 'whom', 'very', 'any', 'no', "couldn't", 'ma', "you've", 'has', "don't", 'yourselves', 'with', 'weren', "should've", 'didn', 'before', 'so', 'who', "wasn't", 'once', 'yours', 'how', 'them', 'our', 'of', "weren't", 'now', 'into', 'during', 'when', 've', 'needn', 'will', 'as', 'had', 'him', 'again', 'mustn', 'against', 'under', 'll', 'after', 'other', 't', 'just', 'from', "that'll", 'through', 'she', 'a', 'we', 'which', 'couldn', 'have', "mightn't", 'ain', "hasn't", 'not', 'then', 'wasn', 'do', 's', 'both', "aren't", "didn't", 'itself', 'does', "needn't", 're', 'theirs', 'most', 'doesn', 'these', "you're", 'that', 'am', 'be', "shouldn't", 'herself', "shan't", 'down', 'those', 'haven', 'isn

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sbgka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Checking removal works on a test text
test_tweet = ["This is a test that stopword removal works."]

def filter_text(tweets):
    for tweet in tweets:
        tokens = tweet.split()
        # Removing each token if part of stop_words
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
        print("filtered_tokens", filtered_tokens)
        
filter_text(test_tweet)

filtered_tokens ['test', 'stopword', 'removal', 'works.']


In [11]:
# Before improvement of stopwords
tweets = tweets_df["tweet"].tolist()
filter_text(tweets)

filtered_tokens ['@switchfoot', 'http://twitpic.com/2y1zl', '-', 'Awww,', "that's", 'bummer.', 'shoulda', 'got', 'David', 'Carr', 'Third', 'Day', 'it.', ';D']
filtered_tokens ['upset', "can't", 'update', 'Facebook', 'texting', 'it...', 'might', 'cry', 'result', 'School', 'today', 'also.', 'Blah!']
filtered_tokens ['@Kenichan', 'dived', 'many', 'times', 'ball.', 'Managed', 'save', '50%', 'rest', 'go', 'bounds']
filtered_tokens ['whole', 'body', 'feels', 'itchy', 'like', 'fire']
filtered_tokens ['@nationwideclass', 'no,', 'behaving', 'all.', "i'm", 'mad.', 'here?', "can't", 'see', 'there.']
filtered_tokens ['@Kwesidei', 'whole', 'crew']
filtered_tokens ['Need', 'hug']
filtered_tokens ['@LOLTrish', 'hey', 'long', 'time', 'see!', 'Yes..', 'Rains', 'bit', ',only', 'bit', 'LOL', ',', "I'm", 'fine', 'thanks', ',', "how's", '?']
filtered_tokens ['@Tatiana_K', 'nope']
filtered_tokens ['@twittera', 'que', 'muera', '?']
filtered_tokens ['spring', 'break', 'plain', 'city...', 'snowing']
filtered_t

The removal of stopwords has reduced the texts. However after analysing the filtered words, it is seen that symbols are still considered, tokens. As such, the removal of symbols would be added to stop_words. Due to the tokenisation making ending words with symbols (eg: "goodbye!") a single token, the removal will only be on tokens with no characters (eg: "!")

In [12]:
symbols = set(string.punctuation)

# Checking removal works on a test text
test_tweet2 = ["This is a @ modified test that stopword removal works."]

def filter_text2(tweets):
    for tweet in tweets:
        tokens = tweet.split()
        # Removing each token if part of symbols
        filtered_tokens = [token for token in tokens if not all(char in symbols for char in token)]
        # Removing each token if part of stop_words
        filtered_tokens = [token for token in filtered_tokens if token.lower() not in stop_words]
        print("filtered_tokens", filtered_tokens)
        
filter_text2(test_tweet2)

filtered_tokens ['modified', 'test', 'stopword', 'removal', 'works.']


In [13]:
# After improvement of stopwords
tweets = tweets_df["tweet"].tolist()
filter_text2(tweets)

filtered_tokens ['@switchfoot', 'http://twitpic.com/2y1zl', 'Awww,', "that's", 'bummer.', 'shoulda', 'got', 'David', 'Carr', 'Third', 'Day', 'it.', ';D']
filtered_tokens ['upset', "can't", 'update', 'Facebook', 'texting', 'it...', 'might', 'cry', 'result', 'School', 'today', 'also.', 'Blah!']
filtered_tokens ['@Kenichan', 'dived', 'many', 'times', 'ball.', 'Managed', 'save', '50%', 'rest', 'go', 'bounds']
filtered_tokens ['whole', 'body', 'feels', 'itchy', 'like', 'fire']
filtered_tokens ['@nationwideclass', 'no,', 'behaving', 'all.', "i'm", 'mad.', 'here?', "can't", 'see', 'there.']
filtered_tokens ['@Kwesidei', 'whole', 'crew']
filtered_tokens ['Need', 'hug']
filtered_tokens ['@LOLTrish', 'hey', 'long', 'time', 'see!', 'Yes..', 'Rains', 'bit', ',only', 'bit', 'LOL', "I'm", 'fine', 'thanks', "how's"]
filtered_tokens ['@Tatiana_K', 'nope']
filtered_tokens ['@twittera', 'que', 'muera']
filtered_tokens ['spring', 'break', 'plain', 'city...', 'snowing']
filtered_tokens ['re-pierced', 'ear

To improve this even further, we will be removing any tokens that:
- start with the symbol "@" as that would represent Tweets that are tagging other users
- start with "https://" or "http://" as it would represent links

In [17]:
# Checking removal works on a test text
test_tweet3 = ["modified test that stopword removal works. It will remove @testtry", 
               "http not removed but https://www.youtube.com/ and http://www.youtube.com/ removed"]

def filter_text3(tweets):
    for tweet in tweets:
        tokens = tweet.split()
        # Removing each token if part of symbols
        filtered_tokens = [token for token in tokens if not (
            all(char in symbols for char in token)
            or token.startswith("@")
            or token.startswith("http://")
            or token.startswith("https://"))]
        # Removing each token if part of stop_words
        filtered_tokens = [token for token in filtered_tokens if token.lower() not in stop_words]
        print("filtered_tokens", filtered_tokens)
        
filter_text3(test_tweet3)

filtered_tokens ['modified', 'test', 'stopword', 'removal', 'works.', 'remove']
filtered_tokens ['http', 'removed', 'removed']


In [18]:
# After improvement of stopwords
tweets = tweets_df["tweet"].tolist()
filter_text3(tweets)

filtered_tokens ['Awww,', "that's", 'bummer.', 'shoulda', 'got', 'David', 'Carr', 'Third', 'Day', 'it.', ';D']
filtered_tokens ['upset', "can't", 'update', 'Facebook', 'texting', 'it...', 'might', 'cry', 'result', 'School', 'today', 'also.', 'Blah!']
filtered_tokens ['dived', 'many', 'times', 'ball.', 'Managed', 'save', '50%', 'rest', 'go', 'bounds']
filtered_tokens ['whole', 'body', 'feels', 'itchy', 'like', 'fire']
filtered_tokens ['no,', 'behaving', 'all.', "i'm", 'mad.', 'here?', "can't", 'see', 'there.']
filtered_tokens ['whole', 'crew']
filtered_tokens ['Need', 'hug']
filtered_tokens ['hey', 'long', 'time', 'see!', 'Yes..', 'Rains', 'bit', ',only', 'bit', 'LOL', "I'm", 'fine', 'thanks', "how's"]
filtered_tokens ['nope']
filtered_tokens ['que', 'muera']
filtered_tokens ['spring', 'break', 'plain', 'city...', 'snowing']
filtered_tokens ['re-pierced', 'ears']
filtered_tokens ['bear', 'watch', 'it.', 'thought', 'UA', 'loss', 'embarrassing']
filtered_tokens ['counts,', 'idk', 'either.

### 2.2 Baseline performance
(writeup not needed)

### 2.3 Classification approach
(writeup not needed)

### 2.4 Coding style
(writeup not needed)

## III. Conclusions

### 3.1 Evaluation

### 3.2 Summary and conclusions

## Temporary reference list
* to use citation generator

- Cynthia Van Hee, Jacobs, G., Emmery, C., Desmet, B., Lefever, E., Verhoeven, B., Guy De Pauw, Daelemans, W. and Hoste, V. (2018). Automatic detection of cyberbullying in social media text. PLOS ONE, [online] 13(10), p.e0203794. doi:https://doi.org/10.1371/journal.pone.0203794.
- Khairy, M., Mahmoud, T.M. and Abd-El-Hafeez, T. (2021). Automatic Detection of Cyberbullying and Abusive Language in Arabic Content on Social Networks: A Survey. Procedia Computer Science, [online] 189, pp.156–166. doi:https://doi.org/10.1016/j.procs.2021.05.080.